In [1]:
!pip install -q ultralytics pycocotools

In [2]:
from ultralytics import YOLO
import wandb
import torch
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import json
import os
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')
wandb.login()

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
def yolo_benchmark(model):
    metrics = model.val(data='/kaggle/input/lung-ct-version-n-512/lung_ct_version_n_512.v2i.yolov8/data_test.yaml')
    print(metrics.box.map)  # map50-95
    print(metrics.box.map50)  # map50
    print(metrics.box.map75)  # map75
    print(metrics.box.maps)  # a list contains map50-95 of each category

def coco_benchmark(model):
    # Load your YOLOv8 model and data
#     model = YOLO('/kaggle/working/runs/detect/train/weights/best.pt')
    data_dir = '/kaggle/input/lung-ct-version-n-512/lung_ct_version_n_512.v2i.yolov8/test'
    val_images_dir = os.path.join(data_dir, 'images')  # Folder containing validation images
    val_annotations_path = "/kaggle/input/lung-ct-version-n-512/lung_ct_version_n_512.v2i.coco/test/_annotations.coco.json"  # COCO-style annotations

    # Load COCO ground truth
    coco_gt = COCO(val_annotations_path)

    # Run inference and collect predictions
    results = []
    for image_id in tqdm(coco_gt.getImgIds(), desc="Running inference"):
        img_info = coco_gt.loadImgs(image_id)[0]
        img_path = os.path.join(val_images_dir, img_info['file_name'])

        # Run inference
        outputs = model(img_path)[0]

        # Format predictions for COCO evaluation
        for box, cls, score in zip(outputs.boxes.xywh, outputs.boxes.cls, outputs.boxes.conf):
            # Convert to COCO format
            x, y, w, h = box.tolist()
            result = {
                "image_id": image_id,
                "category_id": int(cls) + 1,  # COCO classes are 1-indexed
                "bbox": [x - w / 2, y - h / 2, w, h],  # Convert to COCO [x, y, w, h] format
                "score": float(score)
            }
            results.append(result)

    # Save predictions to JSON file
    predictions_path = '/kaggle/working/yolo_predictions.json'
    with open(predictions_path, 'w') as f:
        json.dump(results, f)

    # Load the predictions and evaluate using COCO
    coco_dt = coco_gt.loadRes(predictions_path)
    coco_eval = COCOeval(coco_gt, coco_dt, iouType='bbox')

    # Evaluate AP and AR for different IoU thresholds and area sizes
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()

In [4]:
yolo_model_x = YOLO('yolov8x.pt')

EPOCHS = 50
IMAGE_SIZE = 512

100%|██████████| 131M/131M [00:00<00:00, 384MB/s] 


In [5]:
yolo_model_x_results = yolo_model_x.train(
    data="/kaggle/input/lung-ct-version-n-512/lung_ct_version_n_512.v2i.yolov8/data.yaml",
    epochs=EPOCHS,
    imgsz=IMAGE_SIZE,
    batch=8,
)

Ultralytics 8.3.27 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=/kaggle/input/lung-ct-version-n-512/lung_ct_version_n_512.v2i.yolov8/data.yaml, epochs=50, time=None, patience=100, batch=8, imgsz=512, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save

100%|██████████| 755k/755k [00:00<00:00, 19.4MB/s]
2024-11-04 09:50:52,790	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-11-04 09:50:53,777	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      2320  ultralytics.nn.modules.conv.Conv             [3, 80, 3, 2]                 
  1                  -1  1    115520  ultralytics.nn.modules.conv.Conv             [80, 160, 3, 2]               
  2                  -1  3    436800  ultralytics.nn.modules.block.C2f             [160, 160, 3, True]           
  3                  -1  1    461440  ultralytics.nn.modules.conv.Conv             [160, 320, 3, 2]              
  4                  -1  6   3281920  ultralytics.nn.modules.block.C2f             [320, 320, 6, True]           
  5                  -1  1   1844480  ultralytics.nn.modules.conv.Conv             [320, 640, 3, 2]              
  6                  -1  6  13117440  ultralytics.nn.modules.block.C2f             [640, 640, 6, True]           
  7                  -1  1   3687680  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 13.4MB/s]


AMP: checks passed ✅


...

      50/50      5.73G      1.473     0.7783     0.9289          1        512: 100%|██████████| 303/303 [02:30<00:00,  2.01it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:02<00:00,  3.19it/s]

                   all        117        117      0.836      0.744      0.824      0.441



50 epochs completed in 2.163 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 136.7MB
Optimizer stripped from runs/detect/train/weights/best.pt, 136.7MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.27 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Model summary (fused): 268 layers, 68,124,531 parameters, 0 gradients, 257.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:02<00:00,  2.95it/s]


                   all        117        117      0.808      0.786      0.826      0.467
Speed: 0.2ms preprocess, 18.5ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/train


In [6]:
load_yolo_model_x = YOLO("/kaggle/working/runs/detect/train/weights/best.pt")

In [7]:
yolo_benchmark(load_yolo_model_x)

Ultralytics 8.3.27 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Model summary (fused): 268 layers, 68,124,531 parameters, 0 gradients, 257.4 GFLOPs


val: Scanning /kaggle/input/lung-ct-version-n-512/lung_ct_version_n_512.v2i.yolov8/test/labels... 228 images, 0 backgrounds, 0 corrupt: 100%|██████████| 228/228 [00:00<00:00, 258.63it/s]

val: WARNING ⚠️ Cache directory /kaggle/input/lung-ct-version-n-512/lung_ct_version_n_512.v2i.yolov8/test is not writeable, cache not saved.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:05<00:00,  2.79it/s]


                   all        228        228      0.845      0.815      0.861      0.466
Speed: 0.1ms preprocess, 19.1ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/val
0.4657982736429755
0.861077111158788
0.44633515821953534
[     0.4658]


In [8]:
coco_benchmark(load_yolo_model_x)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


Running inference:   0%|          | 0/228 [00:00<?, ?it/s]


image 1/1 /kaggle/input/lung-ct-version-n-512/lung_ct_version_n_512.v2i.yolov8/test/images/1-3-6-1-4-1-14519-5-2-1-6279-6001-299806338046301317870803017534_191_819_png.rf.0aabf1d0f2b66d7eb9c4cc53ad2dd4e7.jpg: 512x512 (no detections), 25.4ms
Speed: 1.2ms preprocess, 25.4ms inference, 0.7ms postprocess per image at shape (1, 3, 512, 512)

image 1/1 /kaggle/input/lung-ct-version-n-512/lung_ct_version_n_512.v2i.yolov8/test/images/1-3-6-1-4-1-14519-5-2-1-6279-6001-288701997968615460794642979503_56_768_png.rf.0acb5a09390d999c58304b7e5d3f2941.jpg: 512x512 1 nodule, 25.4ms
Speed: 1.0ms preprocess, 25.4ms inference, 1.3ms postprocess per image at shape (1, 3, 512, 512)


Running inference:   1%|          | 2/228 [00:00<00:13, 16.97it/s]


image 1/1 /kaggle/input/lung-ct-version-n-512/lung_ct_version_n_512.v2i.yolov8/test/images/1-3-6-1-4-1-14519-5-2-1-6279-6001-162845309248822193437735868939_501_226_png.rf.0c2c933bdaa745a1bf38488049a17560.jpg: 512x512 1 nodule, 23.3ms
Speed: 1.0ms preprocess, 23.3ms inference, 1.3ms postprocess per image at shape (1, 3, 512, 512)

image 1/1 /kaggle/input/lung-ct-version-n-512/lung_ct_version_n_512.v2i.yolov8/test/images/1-3-6-1-4-1-14519-5-2-1-6279-6001-675543413149938600000570588203_66_1064_png.rf.0ca2f3c8841956ec733fd32164df3fbc.jpg: 512x512 1 nodule, 23.4ms
Speed: 1.0ms preprocess, 23.4ms inference, 1.4ms postprocess per image at shape (1, 3, 512, 512)

image 1/1 /kaggle/input/lung-ct-version-n-512/lung_ct_version_n_512.v2i.yolov8/test/images/1-3-6-1-4-1-14519-5-2-1-6279-6001-194488534645348916700259325236_125_368_png.rf.0e22f2965d930b9c798333b7cc9f8d55.jpg: 512x512 1 nodule, 23.4ms
Speed: 1.1ms preprocess, 23.4ms inference, 1.2ms postprocess per image at shape (1, 3, 512, 512)

ima

...

Running inference:  99%|█████████▉| 226/228 [00:07<00:00, 31.31it/s]


image 1/1 /kaggle/input/lung-ct-version-n-512/lung_ct_version_n_512.v2i.yolov8/test/images/1-3-6-1-4-1-14519-5-2-1-6279-6001-309901913847714156367981722205_52_847_png.rf.fcdf7a1891d15a5189e7fa9a50428af8.jpg: 512x512 (no detections), 23.1ms
Speed: 1.0ms preprocess, 23.1ms inference, 0.5ms postprocess per image at shape (1, 3, 512, 512)

image 1/1 /kaggle/input/lung-ct-version-n-512/lung_ct_version_n_512.v2i.yolov8/test/images/1-3-6-1-4-1-14519-5-2-1-6279-6001-167237290696350215427953159586_60_231_png.rf.f6f9fc8211768a96e99dfd17a858e2f7.jpg: 512x512 1 nodule, 23.1ms
Speed: 0.9ms preprocess, 23.1ms inference, 1.1ms postprocess per image at shape (1, 3, 512, 512)


Running inference: 100%|██████████| 228/228 [00:07<00:00, 31.37it/s]

Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*


DONE (t=0.10s).
Accumulating evaluation results...
DONE (t=0.03s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.438
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.818
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.401
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.441
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.462
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.500
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.528
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.528
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.530
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.525
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= la

In [9]:
!zip -r runs.zip /kaggle/working/

  adding: kaggle/working/ (stored 0%)
  adding: kaggle/working/.virtual_documents/ (stored 0%)
  adding: kaggle/working/yolo_predictions.json (deflated 71%)
  adding: kaggle/working/yolov8x.pt (deflated 8%)
  adding: kaggle/working/runs/ (stored 0%)
  adding: kaggle/working/runs/detect/ (stored 0%)
  adding: kaggle/working/runs/detect/train/ (stored 0%)
  adding: kaggle/working/runs/detect/train/events.out.tfevents.1730713867.15f5951fbc7e.30.0 (deflated 91%)
  adding: kaggle/working/runs/detect/train/val_batch0_pred.jpg (deflated 13%)
  adding: kaggle/working/runs/detect/train/train_batch12121.jpg (deflated 10%)
  adding: kaggle/working/runs/detect/train/labels_correlogram.jpg (deflated 36%)
  adding: kaggle/working/runs/detect/train/train_batch1.jpg (deflated 8%)
  adding: kaggle/working/runs/detect/train/args.yaml (deflated 52%)
  adding: kaggle/working/runs/detect/train/results.csv (deflated 60%)
  adding: kaggle/working/runs/detect/train/val_batch0_labels.jpg (deflated 13%)
  addin